<a href="https://colab.research.google.com/github/rikdantas/Aprendizagem-de-Maquinas/blob/main/IMD1101/Pratica_01/Pratica01_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prática 01 - Utilização de extrator para imagens e uso do k-NN
Nesse notebook serão desenvolvidos os códigos para aplicar o k-NN nas 12 bases geradas pelo HOG e CNN.

Aluno: Paulo Ricardo Dantas

In [8]:
# Importando as bibliotecas
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Baixando os datasets
Como estou separando os notebooks por partes, iremos baixar os datasets resultantes que estão hospedados no Google Drive.

In [4]:
# Criando função para baixar os datasets, assim deixando o código um pouco mais organizado
def baixar_dataset(url, output):
    gdown.download(url, output, quiet=True)

In [6]:
# Baixando datasets do HOG
baixar_dataset('https://drive.google.com/uc?id=1dULiu_42C2HxXn0RrryKOWs0zn9rylug', 'HOG_1.csv')
baixar_dataset('https://drive.google.com/uc?id=1byMlvmIqAtH5r0avtSxoqbEzjKeO1gAm', 'HOG_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1bea8ef7uSiDRknjOxyej6csxdc21MOeh', 'HOG_3.csv')
baixar_dataset('https://drive.google.com/uc?id=1VhY6sc9dj3DyLEI_6DbCwrrke7y2FS9C', 'HOG_4.csv')

# Baixando datasets do VGG16
baixar_dataset('https://drive.google.com/uc?id=1Lig0-UqpGvMQCmV5r5U__yucoocrlbtf', 'VGG16_1.csv')
baixar_dataset('https://drive.google.com/uc?id=1V7_N11eqBnTiM9iKY3vhMVrTzoERrijS', 'VGG16_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1d8spY-o4xBsozdeSbGGlweZrips1m0HC', 'VGG16_3.csv')
baixar_dataset('https://drive.google.com/uc?id=1gaXm38L39O1RRQlUWBX1EIuNb1uz9c7K', 'VGG16_4.csv')

# Baixando datasets do VGG19
baixar_dataset('https://drive.google.com/uc?id=1yjuqQKxXtBqlMetGgD3fyFjDZ3zg3YDS', 'VGG19_1.csv')
baixar_dataset('https://drive.google.com/uc?id=13nmb2kPrv6oVKlhodM-_ierzsMrvWL18', 'VGG19_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1qB2SQXLtgS01FqTrwGYLdOY1pwRqkuXu', 'VGG19_3.csv')
baixar_dataset('https://drive.google.com/uc?id=1sgLqrJdFWQuBtHjmBdUCEYwp-7GEOrDY', 'VGG19_4.csv')

## Importando os datasets
Vamos usar o pandas para importar os datasets para DataFrames. Usaremos os mesmos nomes que foram baixados os arquivos dos datasets. Podemos relacionar os datasets com as configurações como visto na tabela a seguir:

| Nome do dataset | Configuração         |
|------------------|----------------------|
| HOG_1           | HOG_128_16x16       |
| HOG_2           | HOG_128_20x20       |
| HOG_3           | HOG_256_16x16       |
| HOG_4           | HOG_256_20x20       |
| VGG16_1         | CNN_VGG16_128_avg   |
| VGG16_2         | CNN_VGG16_128_max   |
| VGG16_3         | CNN_VGG16_256_avg   |
| VGG16_4         | CNN_VGG16_256_max   |
| VGG19_1         | CNN_VGG19_128_avg   |
| VGG19_2         | CNN_VGG19_128_max   |
| VGG19_3         | CNN_VGG19_256_avg   |
| VGG19_4         | CNN_VGG19_256_max   |


In [7]:
# Importando os datasets do HOG
hog_1 = pd.read_csv('HOG_1.csv', encoding='utf-8')
hog_2 = pd.read_csv('HOG_2.csv', encoding='utf-8')
hog_3 = pd.read_csv('HOG_3.csv', encoding='utf-8')
hog_4 = pd.read_csv('HOG_4.csv', encoding='utf-8')

# Importando os datasets do VGG16
vgg16_1 = pd.read_csv('VGG16_1.csv', encoding='utf-8')
vgg16_2 = pd.read_csv('VGG16_2.csv', encoding='utf-8')
vgg16_3 = pd.read_csv('VGG16_3.csv', encoding='utf-8')
vgg16_4 = pd.read_csv('VGG16_4.csv', encoding='utf-8')

# Importando os datasets do VGG19
vgg19_1 = pd.read_csv('VGG19_1.csv', encoding='utf-8')
vgg19_2 = pd.read_csv('VGG19_2.csv', encoding='utf-8')
vgg19_3 = pd.read_csv('VGG19_3.csv', encoding='utf-8')
vgg19_4 = pd.read_csv('VGG19_4.csv', encoding='utf-8')

## Aplicando o k-NN

### Utilizando holdout
Nessa parte iremos dividir o dataset em um conjunto de 70% para treino e 30% para teste

#### Listando os dataframes para fazer tudo com um loop

In [10]:
# Lista de DataFrames
dataframes = {
    "hog_1": hog_1, "hog_2": hog_2, "hog_3": hog_3, "hog_4": hog_4,
    "vgg16_1": vgg16_1, "vgg16_2": vgg16_2, "vgg16_3": vgg16_3, "vgg16_4": vgg16_4,
    "vgg19_1": vgg19_1, "vgg19_2": vgg19_2, "vgg19_3": vgg19_3, "vgg19_4": vgg19_4,
}

# Configurações
results = {}
random_state = 1

In [11]:
# Loop para aplicar k-NN
for name, df in dataframes.items():
    # A última coluna é o rótulo (classe)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Holdout (70/30)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Testar K de 1 a 10
    accuracies = []
    for k in range(1, 11):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)

    # Armazenar resultados
    results[name] = accuracies

# Exibir resultados
results_df = pd.DataFrame(results, index=[f"K={k}" for k in range(1, 11)])
print(results_df)

         hog_1     hog_2     hog_3     hog_4   vgg16_1   vgg16_2   vgg16_3  \
K=1   0.541667  0.595833  0.525000  0.541667  0.550000  0.575000  0.662500   
K=2   0.529167  0.579167  0.487500  0.491667  0.600000  0.566667  0.654167   
K=3   0.541667  0.558333  0.495833  0.495833  0.595833  0.620833  0.695833   
K=4   0.525000  0.562500  0.504167  0.500000  0.566667  0.608333  0.666667   
K=5   0.520833  0.583333  0.508333  0.491667  0.608333  0.658333  0.654167   
K=6   0.533333  0.570833  0.520833  0.487500  0.587500  0.641667  0.654167   
K=7   0.554167  0.587500  0.516667  0.475000  0.612500  0.654167  0.666667   
K=8   0.541667  0.575000  0.516667  0.504167  0.629167  0.629167  0.654167   
K=9   0.562500  0.587500  0.516667  0.504167  0.625000  0.637500  0.691667   
K=10  0.545833  0.566667  0.516667  0.520833  0.645833  0.650000  0.670833   

       vgg16_4   vgg19_1   vgg19_2   vgg19_3   vgg19_4  
K=1   0.525000  0.583333  0.591667  0.629167  0.541667  
K=2   0.537500  0.583333  0